In [89]:
import array, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import poisson
import statsmodels.api as sm

In [90]:
import patsy, cython

In [91]:
pd.set_option('display.max_rows', None)
tabela = pd.read_csv("WorldCupMatches.csv", encoding='utf-8')
chaveamento = pd.read_csv("fixtures_2022.csv", encoding = 'utf-8')

In [92]:
to_delete = ["Match Number","Date", "Location"]
chaveamento.drop_duplicates(inplace=True)
for i in to_delete:
    chaveamento.drop(i,axis=1,inplace=True)

In [93]:
# Manipulando a tabela dos jogos. Foram excluídos confrontos eliminatórios
# pois eles seram decididos através da IA.
chaveamento = pd.DataFrame(data = [chaveamento["Round Number"], chaveamento["Group"], chaveamento["Home Team"], chaveamento["Away Team"]]).transpose()
replacement = ['EF1','EF2','EF3','EF4','EF5','EF6','EF7','EF8','QF1','QF2','QF3','QF4','SF1','SF2','3rd','Finals']
tam = len(chaveamento["Group"][48:])
for i in range(tam):
    chaveamento['Group'][48+i] = replacement[i]
chaveamento['Round Number'][62] = 'Third Place'
chaveamento['Home Team'][48:] = " "
chaveamento['Away Team'][48:] = " "

In [94]:
# Manipulação feita sobre a tabela de dados históricos da entrega 2.
to_delete = ["Datetime","Stadium","City","Referee","Assistant 1", "Assistant 2","MatchID"]
tabela.drop_duplicates(inplace=True)
for i in to_delete:
    tabela.drop(i,axis=1,inplace=True)

# Faz a limpeza/troca de nomes de times com erros
tabela["Home Team Name"] = tabela["Home Team Name"].str.replace('rn">', '', regex=True)
tabela["Away Team Name"] = tabela["Away Team Name"].str.replace('rn">', '', regex=True)

names_tobe_replaced = ["C�te d'Ivoire", "Germany FR", "German DR", "IR Iran", "Zaire", "Dutch East Indies","Republic of Ireland"]
names_to_replace = ["Côte d'Ivoire", "Germany", "Germany", "Iran", "Congo DR", "Indonesia", "Ireland"]

for i,j in zip(names_tobe_replaced, names_to_replace):
    tabela["Home Team Name"] = tabela["Home Team Name"].replace([i],[j])
    tabela["Away Team Name"] = tabela["Away Team Name"].replace([i],[j])


In [95]:
# Fazer a média histórica de gols feitos/sofridos pelos times

home = tabela[['Home Team Name', 'Home Team Goals', 'Away Team Goals']]
away = tabela[['Away Team Name', 'Home Team Goals', 'Away Team Goals']]

home = home.rename(columns={'Home Team Name': 'Team', 'Home Team Goals': 'Goals Scored', 'Away Team Goals': 'Goals Conceded'})
away = away.rename(columns={'Away Team Name': 'Team', 'Home Team Goals': 'Goals Conceded', 'Away Team Goals': 'Goals Scored'})

gsgc = pd.concat([home, away], ignore_index = True).groupby(['Team']).mean()

In [96]:
# Função preditora básica
def result(home_team, away_team):
    if home_team in gsgc.index and away_team in gsgc.index:
        lambda_home = gsgc.at[home_team,'Goals Scored']*gsgc.at[away_team,'Goals Conceded']
        lambda_away = gsgc.at[away_team,'Goals Scored']*gsgc.at[home_team,'Goals Conceded']
        home_win, away_win, draw = 0, 0, 0
        for x in range(0,9):
            for y in range(0,6):
                p = poisson.pmf(x, lambda_home)*poisson.pmf(y, lambda_away)
                if x == y:
                    draw += p
                elif x > y:
                    home_win += p
                else:
                    away_win += p
        home_pts = 3*home_win + draw
        away_pts = 3*away_win + draw
        return (home_pts, away_pts)
    else:
        return (0, 0)

In [97]:
# Processo de criação do dataframe com os resultados finais da fase de grupos
groups = ['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H']
groups_phase = {}
for group in groups:
    groups_phase[format(group)] = chaveamento[chaveamento['Group'] == group]

In [98]:
# Processo de criação do dataframe com os resultados finais da fase de grupos
dicte = {'Group A':[], 'Group B':[], 'Group C':[], 'Group D':[], 'Group E':[], 'Group F':[], 'Group G':[], 'Group H':[],}
for group in groups:
  k = groups_phase[group]['Home Team'].unique().tolist()
  dicte[group].append(k)

In [99]:
# Processo de criação do dataframe com os resultados finais da fase de grupos
dfdf = pd.DataFrame(dicte)
gnc = dfdf.explode(groups, ignore_index = True)

impares = [2*i + 1 for i in range(0,8)]
letras = ["A", "B", "C", "D", "E", "F", "G", "H"]

j = 0
for i in impares:
  gnc.insert(i,f"Pts {letras[j]}",[0,0,0,0], True)
  j += 1

In [100]:
# Manipulação do dataframe gnc afim de arrumar os índices dos times que estavam em posições erradas (por algum motivo)
gnc['Group B'][2], gnc['Group B'][3] = 'Iran', 'Wales'
gnc['Group C'][0], gnc['Group C'][1], gnc['Group C'][2], gnc['Group C'][3] = 'Saudi Arabia', 'Poland', 'Argentina', 'Mexico'
gnc['Group D'][0], gnc['Group D'][1], gnc['Group D'][2], gnc['Group D'][3] = 'Australia', 'Tunisia', 'France', 'Denmark'
gnc['Group E'][1], gnc['Group E'][2] = 'Japan', 'Spain'
gnc['Group F'][0], gnc['Group F'][1], gnc['Group F'][2], gnc['Group F'][3] = 'Belgium', 'Croatia', 'Canada', 'Morocco'
gnc['Group G'][0], gnc['Group G'][2], gnc['Group G'][3] = 'Cameroon', 'Serbia', 'Switzerland'
gnc['Group H'][0], gnc['Group H'][1], gnc['Group H'][2], gnc['Group H'][3] = 'Korea Republic', 'Ghana', 'Portugal', 'Uruguay'
gnc2 = gnc.copy()

<ipython-input-100-706bda88c403>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnc['Group B'][2], gnc['Group B'][3] = 'Iran', 'Wales'
<ipython-input-100-706bda88c403>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnc['Group C'][0], gnc['Group C'][1], gnc['Group C'][2], gnc['Group C'][3] = 'Saudi Arabia', 'Poland', 'Argentina', 'Mexico'
<ipython-input-100-706bda88c403>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnc['Group D'

In [101]:
# Processo de criação do dataframe com os resultados finais da fase de grupos
grp_phase = chaveamento[:48].sort_values('Group')
resultados = []
grp_phase.insert(3,"Home Pts"," ")
grp_phase.insert(4,"Away Pts"," ")

for row in grp_phase.index:
    a =  grp_phase['Home Team'][row]
    b =  grp_phase['Away Team'][row]
    resultado = result(a,b)
    grp_phase['Home Pts'][row] = resultado[0]
    grp_phase['Away Pts'][row] = resultado[1]

In [102]:
# Processo de criação do dataframe com os resultados finais da fase de grupos
soma_pts = []
times = pd.DataFrame(grp_phase['Home Team'].append(grp_phase['Away Team'], ignore_index=True))
times = times[0].unique()
groups_pts = ['Pts A', 'Pts B', 'Pts C', 'Pts D', 'Pts E', 'Pts F', 'Pts G', 'Pts H']

for time in times:
    tmp3 = sum(grp_phase['Home Pts'][grp_phase['Home Team'] == time]) + sum(grp_phase['Away Pts'][grp_phase['Away Team'] == time])
    soma_pts.append(tmp3)

In [103]:
# Criação do dataframe com os resultados finais da fase de grupos

arr_pts = [round(num, 0) for num in soma_pts] 
j = 0
for i in groups_pts:
    for z in range(0,4):
        var_temp = arr_pts[j]
        gnc2[i][z] = var_temp 
        j += 1

<ipython-input-103-aab2fc41b73b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnc2[i][z] = var_temp


In [104]:
# Manipulação do dataframe gnc2 com resultados finais de cada grupo, simulados pelo modelo
grpA = pd.DataFrame(data = [gnc2['Group A'], gnc2['Pts A']]).transpose().sort_values(['Pts A'], ascending =  False)
grpB = pd.DataFrame(data = [gnc2['Group B'], gnc2['Pts B']]).transpose().sort_values(['Pts B'], ascending =  False)
grpC = pd.DataFrame(data = [gnc2['Group C'], gnc2['Pts C']]).transpose().sort_values(['Pts C'], ascending =  False)
grpD = pd.DataFrame(data = [gnc2['Group D'], gnc2['Pts D']]).transpose().sort_values(['Pts D'], ascending =  False)
grpE = pd.DataFrame(data = [gnc2['Group E'], gnc2['Pts E']]).transpose().sort_values(['Pts E'], ascending =  False)
grpF = pd.DataFrame(data = [gnc2['Group F'], gnc2['Pts F']]).transpose().sort_values(['Pts F'], ascending =  False)
grpG = pd.DataFrame(data = [gnc2['Group G'], gnc2['Pts G']]).transpose().sort_values(['Pts G'], ascending =  False)
grpH = pd.DataFrame(data = [gnc2['Group H'], gnc2['Pts H']]).transpose().sort_values(['Pts H'], ascending =  False)

In [105]:
# Manipulação do dataframe gnc2 com resultados finais de cada grupo, simulados pelo modelo
grpA.sort_values(by=['Pts A','Group A'], ascending = False, ignore_index = True)
grpB.sort_values(by=['Pts B','Group B'], ascending = False, ignore_index = True)
grpC.sort_values(by=['Pts C','Group C'], ascending = False, ignore_index = True)
grpD.sort_values(by=['Pts D','Group D'], ascending = False, ignore_index = True)
grpE.sort_values(by=['Pts E','Group E'], ascending = False, ignore_index = True)
grpF.sort_values(by=['Pts F','Group F'], ascending = False, ignore_index = True)
grpG.sort_values(by=['Pts G','Group G'], ascending = False, ignore_index = True)
grpH.sort_values(by=['Pts H','Group H'], ascending = False, ignore_index = True)
grpA = grpA.rename(index = {2:'1st',1:'2nd',3:'3rd',0:'4th'})
grpB = grpB.rename(index = {0:'1st',3:'2nd',1:'3rd',2:'4th'})
grpC = grpC.rename(index = {2:'1st',1:'2nd',3:'3rd',0:'4th'})
grpD = grpD.rename(index = {2:'1st',3:'2nd',1:'3rd',0:'4th'})
grpE = grpE.rename(index = {0:'1st',2:'2nd',1:'3rd',3:'4th'})
grpF = grpF.rename(index = {1:'1st',0:'2nd',3:'3rd',2:'4th'})
grpG = grpG.rename(index = {1:'1st',3:'2nd',2:'3rd',0:'4th'})
grpH = grpH.rename(index = {2:'1st',3:'2nd',1:'3rd',0:'4th'})

In [158]:
grpA

,Group A,Pts A
1st,Netherlands,4
2nd,Senegal,2
3rd,Ecuador,2
4th,Qatar,0


In [159]:
grpB

,Group B,Pts B
1st,England,6
2nd,Wales,5
3rd,USA,3
4th,Iran,2


In [160]:
grpC

,Group C,Pts C
1st,Argentina,7
2nd,Poland,5
3rd,Mexico,3
4th,Saudi Arabia,1


In [161]:
grpD

,Group D,Pts D
1st,France,7
2nd,Denmark,5
3rd,Tunisia,3
4th,Australia,2


In [162]:
grpE

,Group E,Pts E
1st,Germany,6
2nd,Spain,5
3rd,Japan,3
4th,Costa Rica,3


In [163]:
grpF

,Group F,Pts F
1st,Croatia,7
2nd,Belgium,6
3rd,Morocco,4
4th,Canada,0


In [164]:
grpG

,Group G,Pts G
1st,Brazil,7
2nd,Switzerland,4
3rd,Serbia,3
4th,Cameroon,2


In [165]:
grpH

,Group H,Pts H
1st,Portugal,6
2nd,Uruguay,5
3rd,Ghana,4
4th,Korea Republic,2


In [114]:
#Eliminatórias - Oitavas

EF = chaveamento[48:56].copy()
EF.insert(3, "Winner", " ")
QF = chaveamento[56:60].copy()
QF.insert(3, "Winner", " ")
SF = chaveamento[60:62].copy()
SF.insert(3, "Winner", " ")
FF = chaveamento[62:64].copy()
FF.insert(3,'Winner', ' ')

In [170]:
#Manipulação do dataframe com times que avançaram para as eliminatórias
modular = pd.concat([grpA, grpB, grpC, grpD, grpE, grpF, grpG, grpH], axis = 1, ignore_index=False)
modular = modular.iloc[0:2]
for g in groups_pts:
  modular.drop(g, axis=1,inplace=True)

In [171]:
#Manipulação do dataframe com times que avançaram para as eliminatórias
times_avancam = []
for group in groups:
  times_avancam.append(modular[group].transpose().tolist())

times_avancam = sum(times_avancam,[])
pri_lugares = times_avancam[0:len(times_avancam):2]
seg_lugares = ['Wales','Senegal','Denmark','Poland','Belgium','Spain','Uruguay','Switzerland']
pri_lugares = pri_lugares[0:len(pri_lugares):2] + pri_lugares[1:len(pri_lugares):2]
seg_lugares = seg_lugares[0:len(pri_lugares):2] + seg_lugares[1:len(pri_lugares):2]


EF['Home Team'] = pri_lugares
EF['Away Team'] = seg_lugares

In [172]:
for row in EF.index:
    a =  EF['Home Team'][row]
    b =  EF['Away Team'][row]
    res1, res2 = result(a,b)
    if res1 >= res2:
      EF['Winner'][row] = a
    else:
      EF['Winner'][row] = b

EF

,Round Number,Group,Home Team,Winner,Away Team
48,Round of 16,EF1,Netherlands,Netherlands,Wales
49,Round of 16,EF2,Argentina,Argentina,Denmark
50,Round of 16,EF3,Germany,Germany,Belgium
51,Round of 16,EF4,Brazil,Brazil,Uruguay
52,Round of 16,EF5,England,England,Senegal
53,Round of 16,EF6,France,France,Poland
54,Round of 16,EF7,Croatia,Spain,Spain
55,Round of 16,EF8,Portugal,Portugal,Switzerland


In [153]:
# Eliminatórias - Quartas
ganhadores_EF = EF['Winner'].tolist()
pri_EF = ganhadores_EF[0:len(ganhadores_EF):2]
seg_EF = ganhadores_EF[1:len(ganhadores_EF):2]
QF['Home Team'] = pri_EF
QF['Away Team'] = seg_EF

for row in QF.index:
    a =  QF['Home Team'][row]
    b =  QF['Away Team'][row]
    res1, res2 = result(a,b)
    if res1 >= res2:
      QF['Winner'][row] = a
    else:
      QF['Winner'][row] = b

QF

,Round Number,Group,Home Team,Winner,Away Team
56,Quarter Finals,QF1,Netherlands,Netherlands,Argentina
57,Quarter Finals,QF2,Germany,Brazil,Brazil
58,Quarter Finals,QF3,England,France,France
59,Quarter Finals,QF4,Spain,Portugal,Portugal


In [166]:
# Eliminatórias - Semis
ganhadores_QF = QF['Winner'].tolist()
pri_QF = ganhadores_QF[0:len(ganhadores_QF):2]
seg_QF = ganhadores_QF[1:len(ganhadores_QF):2]
SF['Home Team'] = pri_QF
SF['Away Team'] = seg_QF

for row in SF.index:
    a =  SF['Home Team'][row]
    b =  SF['Away Team'][row]
    res1, res2 = result(a,b)
    if res1 >= res2:
      SF['Winner'][row] = a
    else:
      SF['Winner'][row] = b

SF

,Round Number,Group,Home Team,Winner,Away Team
60,Semi Finals,SF1,Argentina,Argentina,Croatia
61,Semi Finals,SF2,France,France,Morocco


In [167]:
# Eliminatórias - Finais
#FF.insert(3,'Winner', ' ')
FF['Home Team'] = ['France', 'Netherlands']
FF['Away Team'] = ['Germany', 'Brazil']

for row in FF.index:
    a =  FF['Home Team'][row]
    b =  FF['Away Team'][row]
    res1, res2 = result(a,b)
    if res1 >= res2:
      FF['Winner'][row] = a
    else:
      FF['Winner'][row] = b

FF

,Round Number,Group,Home Team,Winner,Away Team
62,Third Place,3rd,Argentina,France,France
63,Finals,Finals,Croatia,Croatia,Morocco
